# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, roadName):
    point = Point(lng, lat, srid=4326)
    roads = Road.objects.filter(company=company,name__icontains=roadName).annotate(distance=Distance(point, 'path')).order_by('distance')
    road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [12]:
!cat .env

cat: .env: No such file or directory


Escolhendo nome do arquivo Excel para carregar o Inventário

In [15]:
filename='Sinalização Horizontal - ERS 446 - CORRIGIDO'

Escolhendo a aba da planilha do Excel

In [16]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Testando planilha carregada do Excel

In [17]:
print(ws['A1'].value)

km


Carregando os valores no script

In [18]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass

        
    values.append(obj)

Teste de valores importados da planilha

In [19]:
len(values),values[41]['Longitude_Inicial']

(198, None)

In [62]:
# for a in values:
#     a['Latitude_Inicial']=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
#     a['Longitude_Inicial']=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]

Definindo valores da importação

In [20]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Sinalização Horizontal
occurrence_type=OccurrenceType.objects.get(uuid='9b1718bc-6046-4d7a-996a-4cae53d320e0')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Sinalização Horizontal - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [29]:
values[5]['Longitude_Inicial']

'-51467813'

In [21]:
print('Km inicial',' | ','km',' | ','km final')
for i,a in enumerate(tqdm(values)):
    a['Longitude_Inicial']=str(a['Longitude_Inicial']).replace(',','').replace('.','')
    a['Latitude_Inicial']=str(a['Latitude_Inicial']).replace(',','').replace('.','')

Km inicial  |  km  |  km final


  0%|          | 0/198 [00:00<?, ?it/s]

Criando itens de Inventário

In [30]:
objects=[]


for index, a in enumerate(tqdm(values)):

    long=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
    lat=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
    pista = 'Pista Dupla' if a['Pista'] == 'DUPLA' else 'Pista Simples'
    road,_,point,km =point_to_km(float(long),float(lat),'446')
    if a['Longitude_Final'] and a['Latitude_Final']:
        long_end=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
        lat_end=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
        _,_,_,end_km =point_to_km(float(long_end),float(lat_end),road)
    if '446' not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")
    try:
        
        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', pista)),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km=float(end_km) if a['Longitude_Final'] and a['Latitude_Final'] else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'extension': a['Extensão\n(m)'],
                        'bd': True if a['Largura_BD'] is not None else False,
                        'largurabd': a['Largura_BD'] if a['Largura_BD'] is not None else None,
                        'corbd': a['Cor_BD'] if a['Cor_BD'] is not None else None,
                        'm1bd': a['BD-M1'] if a['BD-M1'] is not None else None,
                        'm2bd': a['BD-M2'] if a['BD-M2'] is not None else None,
                        'm3bd': a['BD-M3'] if a['BD-M3'] is not None else None,
                        'm4bd': a['BD-M4'] if a['BD-M4'] is not None else None,
                        'm5bd': a['BD-M5'] if a['BD-M5'] is not None else None,
                        'm6bd': a['BD-M6'] if a['BD-M6'] is not None else None,
                        'm7bd': a['BD-M7'] if a['BD-M7'] is not None else None,
                        'm8bd': a['BD-M8'] if a['BD-M8'] is not None else None,
                        'm9bd': a['BD-M9'] if a['BD-M9'] is not None else None,
                        'm10bd': a['BD-M10'] if a['BD-M10'] is not None else None,
                        'mediabd': a['Média_BD'] if a['Média_BD'] is not None else None,
                        'e1': True if a['Largura_E_1'] is not None else False,
                        'largurae1': a['Largura_E_1'] if a['Largura_E_1'] is not None else None,
                        'core1': a['Cor_E_1'] if a['Cor_E_1'] is not None else None,
                        'm1e1': a['E_1-M_1'] if a['E_1-M_1'] is not None else None,
                        'm2e1': a['E_1-M_2'] if a['E_1-M_2'] is not None else None,
                        'm3e1': a['E_1-M_3'] if a['E_1-M_3'] is not None else None,
                        'm4e1': a['E_1-M_4'] if a['E_1-M_4'] is not None else None,
                        'm5e1': a['E_1-M_5'] if a['E_1-M_5'] is not None else None,
                        'm6e1': a['E_1-M_6'] if a['E_1-M_6'] is not None else None,
                        'm7e1': a['E_1-M_7'] if a['E_1-M_7'] is not None else None,
                        'm8e1': a['E_1-M_8'] if a['E_1-M_8'] is not None else None,
                        'm9e1': a['E_1-M_9'] if a['E_1-M_9'] is not None else None,
                        'm10e1': a['E_1-M_10'] if a['E_1-M_10'] is not None else None,
                        'mediae1': a['Média_E_1'] if a['Média_E_1'] is not None else None,
                        'e2': True if a['Largura_E_2'] is not None else False,
                        'largurae2': a['Largura_E_2'] if a['Largura_E_2'] is not None else None,
                        'core2': a['Cor_E_2'] if a['Cor_E_2'] is not None else None,
                        'm1e2': a['E_2-M_1'] if a['E_2-M_1'] is not None else None,
                        'm2e2': a['E_2-M_2'] if a['E_2-M_2'] is not None else None,
                        'm3e2': a['E_2-M_3'] if a['E_2-M_3'] is not None else None,
                        'm4e2': a['E_2-M_4'] if a['E_2-M_4'] is not None else None,
                        'm5e2': a['E_2-M_5'] if a['E_2-M_5'] is not None else None,
                        'm6e2': a['E_2-M_6'] if a['E_2-M_6'] is not None else None,
                        'm7e2': a['E_2-M_7'] if a['E_2-M_7'] is not None else None,
                        'm8e2': a['E_2-M_8'] if a['E_2-M_8'] is not None else None,
                        'm9e2': a['E_2-M_9'] if a['E_2-M_9'] is not None else None,
                        'm10e2': a['E_2-M_10'] if a['E_2-M_10'] is not None else None,
                        'mediae2': a['Média_E_2'] if a['Média_E_2'] is not None else None,
                        'be':  True if a['Largura_BE'] is not None else False,
                        'largurabe': a['Largura_BE'] if a['Largura_BE'] is not None else None,
                        'corbe': a['Cor_BE'] if a['Cor_BE'] is not None else None,
                        'm1be': a['BE-M1'] if a['BE-M1'] is not None else None,
                        'm2be': a['BE-M2'] if a['BE-M2'] is not None else None,
                        'm3be': a['BE-M3'] if a['BE-M3'] is not None else None,
                        'm4be': a['BE-M4'] if a['BE-M4'] is not None else None,
                        'm5be': a['BE-M5'] if a['BE-M5'] is not None else None,
                        'm6be': a['BE-M6'] if a['BE-M6'] is not None else None,
                        'm7be': a['BE-M7'] if a['BE-M7'] is not None else None,
                        'm8be': a['BE-M8'] if a['BE-M8'] is not None else None,
                        'm9be': a['BE-M9'] if a['BE-M9'] is not None else None,
                        'm10be': a['BE-M10'] if a['BE-M10'] is not None else None,
                        'mediabe': a['Média_BE'] if a['Média_BE'] is not None else None,
                        
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/198 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'Non.e'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [32]:
len(objects),objects[5][0].__dict__

(7,
 {'_state': <django.db.models.base.ModelState at 0x7f5cc6485e90>,
  'uuid': UUID('b4ffc689-5769-41b0-9b82-389aa3cb7648'),
  'number': '',
  'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
  'road_name': 'ERS-446',
  'road_id': 2061,
  'km': 13.516,
  'end_km': None,
  'km_reference': None,
  'project_km': 0,
  'project_end_km': None,
  'point': <Point object at 0x7f5cc65b90a0>,
  'direction': '0',
  'lane': '2',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('0edd2ecc-8d5e-4444-898b-0069a3e9c354'),
  'occurrence_type_id': UUID('9b1718bc-6046-4d7a-996a-4cae53d320e0'),
  'form_data': {'extension': None,
   'bd': True,
   'largurabd': 0.15,
   'corbd': 'Branca',
   'm1bd': 0,
   'm2bd': 0,
   'm3bd': 0,
   'm4bd': 0,
   'm5bd': 0,
   'm6bd': 0,
   'm7bd': 0,
   'm8bd': 0,
   'm9bd': 0,
   'm10bd': 0,
   'mediabd': 0,
   'e1': True,
   'largurae1': 0.13,
   'core1': 'Amarela',
   'm1e1':

In [33]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/7 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24879
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24880
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24881
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24882
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24883
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24884
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2061: ERS-446
CSG-Inv-2023.24885


Anexando as imagens

In [34]:
for reporting, images in tqdm(objects):
    for image in reversed(images):
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/7 [00:00<?, ?it/s]